# AI Engineering fundamentals 2🔥
## Here, take a deeper dive into neural nets with a focus on the more complex ones

## SECTION 1

<img src="http://cocl.us/general_neural_network" alt="Neural Network General" width="600px">


In [2]:
n = 2 # number of inputs
num_hidden_layers = 2 # number of hidden layers
m = [2, 2] # number of nodes in each hidden layer
num_nodes_output = 1 # number of nodes in the output layer

##### let us now define the strucutre of ANY neural network more clearly, using the below code
##### In order to be able to initialize the weights and the biases to random numbers, we will need to import the Numpy library.
###### Effectively , our objective is to be able to initialize the weights and the biases pertianig to any network of any number of hidden layers and numbers of nodes in eac layer. But let us see hwo that works and see how we can make our code re-usable!

In [3]:
import numpy as np # import the Numpy library

num_nodes_previous = n # number of nodes in the previous layer

network = {} # initialize network an an empty dictionary

# loop through each layer and randomly initialize the weights and biases associated with each node
# notice how we are adding 1 to the number of hidden layers in order to include the output layer
for layer in range(num_hidden_layers + 1): 
    
    # determine name of layer
    if layer == num_hidden_layers:
        layer_name = 'output'
        num_nodes = num_nodes_output
    else:
        layer_name = 'layer_{}'.format(layer + 1)
        num_nodes = m[layer]
    
    # initialize weights and biases associated with each node in the current layer
    network[layer_name] = {}
    for node in range(num_nodes):
        node_name = 'node_{}'.format(node+1)
        network[layer_name][node_name] = {
            'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
            'bias': np.around(np.random.uniform(size=1), decimals=2),
        }
    
    num_nodes_previous = num_nodes
    
print(network) # print network

{'layer_1': {'node_1': {'weights': array([0.16, 0.78]), 'bias': array([0.74])}, 'node_2': {'weights': array([0.41, 0.82]), 'bias': array([0.77])}}, 'layer_2': {'node_1': {'weights': array([0.49, 0.12]), 'bias': array([0.24])}, 'node_2': {'weights': array([0.63, 0.09]), 'bias': array([0.8])}}, 'output': {'node_1': {'weights': array([0.1 , 0.17]), 'bias': array([0.23])}}}


In [4]:
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):
    
    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}
    
    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):
        
        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer] 
        
        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }
    
        num_nodes_previous = num_nodes

    return network # return the network

#### On the above, we initialize_netwrok function but now let us create a netwrok that can:
1. Take 5 inputs
2. Has 3 hidden layers
3. Has 3 nodes in the first layer, 2 nodes in the 2nd layer, and 3 nodes in the third layer
4. Has 1 node in the output layer
##### We will call this network, small_network


In [5]:
import numpy as np

# Define the network parameters
num_inputs = 5
num_hidden_layers = 3
num_nodes_hidden = [3, 2, 3]  # Number of nodes in each hidden layer
num_nodes_output = 1

# Create the network using the initialize_network function
small_network = initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output)

print(small_network)

{'layer_1': {'node_1': {'weights': array([0.48, 0.72, 0.04, 0.4 , 0.09]), 'bias': array([0.14])}, 'node_2': {'weights': array([0.38, 0.42, 0.49, 0.89, 0.92]), 'bias': array([0.8])}, 'node_3': {'weights': array([0.93, 0.74, 0.84, 0.71, 0.13]), 'bias': array([0.44])}}, 'layer_2': {'node_1': {'weights': array([0.34, 1.  , 0.88]), 'bias': array([0.08])}, 'node_2': {'weights': array([0.43, 0.52, 0.96]), 'bias': array([0.67])}}, 'layer_3': {'node_1': {'weights': array([0.79, 0.21]), 'bias': array([0.84])}, 'node_2': {'weights': array([0.05, 0.28]), 'bias': array([0.99])}, 'node_3': {'weights': array([0.2 , 0.39]), 'bias': array([0.3])}}, 'output': {'node_1': {'weights': array([0.2 , 0.41, 0.39]), 'bias': array([0.54])}}}


## SECTION 2

### Let us compute the weighted sum at each node

In [6]:
def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs * weights) + bias

In [7]:
# Generate 5 inputs that we can fee to small_network,
from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

The inputs to the network are [0.15 0.74 0.26 0.53 0.01]


In [9]:
## Let us now compute the weighed sum

# Extract the weights and bias for the first node in the first hidden layer
weights = small_network['layer_1']['node_1']['weights']
bias = small_network['layer_1']['node_1']['bias']

# Compute the weighted sum
weighted_sum = compute_weighted_sum(inputs, weights, bias)

print("The weighted sum at the first node in the first hidden layer is:", weighted_sum)

The weighted sum at the first node in the first hidden layer is: [0.9681]


## SECTION 3
### We now copmute the node action (or activation function in full)

In [10]:
#Each node is simply a non-linear tranformation of the weighted sum. We use activation functions for this mapping. Let's use the sigmoid function as the activation function here. 
#So let's define a function that takes a weighted sum as input and returns the non-linear transformation of the input using the sigmoid function.

def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

In [11]:
# To compute the output of the first node in the first hidden layer, we will need to use the node_activation function with the calculated weighted sum from the previous step.

# Compute the node activation
node_output = node_activation(weighted_sum)

print("The output of the first node in the first hidden layer is:", node_output)


The output of the first node in the first hidden layer is: [0.72474063]


#### Forward Propagation
##### The final piece of building a neural network that can perform predictions is to put everything together. So let's create a function that applies the compute_weighted_sum and node_activation functions to each node in the network and propagates the data all the way to the output layer and outputs a prediction for each node in the output layer.%%Exciting 🙂%%.

##### METHODOLOGY in achieving the above
1. Start with the input layer as the input to the first hidden layer.
2. Compute the weighted sum at the nodes of the current layer.
3. Compute the output of the nodes of the current layer.
4. Set the output of the current layer to be the input to the next layer.
5. Move to the next layer in the network.
6. Repeat steps 2 - 4 until we compute the output of the output layer.

In [12]:
def forward_propagate(network, inputs):
    
    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer
    
    for layer in network:
        
        layer_data = network[layer]
        
        layer_outputs = [] 
        for layer_node in layer_data:
        
            node_data = layer_data[layer_node]
        
            # compute the weighted sum and the output of each node at the same time 
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))
            
        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))
    
        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

In [13]:
# Now, using the forward_propagate function to compute the prediction of our small network, we get the following

# Make a prediction using the small_network
prediction = forward_propagate(small_network, inputs)

print("The prediction of the small network is:", prediction)

The outputs of the nodes in hidden layer number 1 is [0.7247, 0.8552, 0.8485]
The outputs of the nodes in hidden layer number 2 is [0.8731, 0.9039]
The outputs of the nodes in hidden layer number 3 is [0.8481, 0.7836, 0.6958]
The prediction of the small network is: [0.7862]


## Conclusion
### Obviously, the above gave us a bit of a glimpse into what potentially lies ahead of this AI engineering journey indeed